In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
ratings = pd.read_csv('ratings.csv')

In [9]:
def train_test_split_user(ratings, test_size=0.2):
    train_list = []
    test_list = []

    # Get all unique users
    for user_id in ratings['userId'].unique():
        # Filter the dataset to include only rows corresponding to curr user
        user_data = ratings[ratings['userId'] == user_id]
        # Check if user has rated more than 5 movies to meaningfully split data into train and test
        # Ex. user with 10 ratings -> 8 train, 2 test, but user with 3 ratings -> 2 train, 1 test
            # In the latter case, we would not have enough data to train the model
        # If user has rated more than 5 movies, split the data into train and test
        # Else, include all data in train
        if len(user_data) >= 5:
            train_data, test_data = train_test_split(user_data, test_size=test_size, random_state=42)
            train_list.append(train_data)
            test_list.append(test_data)
        else:
            train_list.append(user_data)
    
    # Combine training and testing data for all users into train and test
    train = pd.concat(train_list)
    test = pd.concat(test_list)
    return train, test

train_data, test_data = train_test_split_user(ratings)

# We don't need timestamp column
train_data = train_data.drop(columns=['timestamp'])
test_data = test_data.drop(columns=['timestamp'])



In [10]:
train_data

,userId,movieId,rating
55,1,1031,5.0
230,1,4006,4.0
69,1,1197,5.0
168,1,2596,5.0
109,1,1777,4.0
...,...,...,...
100629,610,103219,3.5
100664,610,107436,3.0
100828,610,163981,3.5
100394,610,70946,5.0


In [11]:
test_data

,userId,movieId,rating
219,1,3578,5.0
66,1,1127,4.0
9,1,157,5.0
170,1,2617,2.0
15,1,260,5.0
...,...,...,...
99733,610,3552,3.0
99830,610,4437,5.0
99820,610,4310,2.5
100531,610,90600,3.5
